# Example of use of eBird2LaTeX python module

In this example, we will see how to use the `e2L` package and produce a checklist pdf. This script is a good place to start learning about the package.


## 1. Setup

First, let's import the dependancies as well as the `e2L` module [from the github repository](https://github.com/Zoziologie/ebird2latex)


In [1]:
import os
import json
!wget --no-cache --backups=1 "https://raw.githubusercontent.com/Zoziologie/ebird2latex/master/e2L.py"
import e2L
!pip install cssselect # not sure why this need to be installed manually
import cssselect
# Download the tex template
!mkdir assets/
!wget --no-cache --backups=1 "https://raw.githubusercontent.com/Zoziologie/ebird2latex/master/assets/templateLaTeX.tex" -P assets/

--2023-11-14 22:53:56--  https://raw.githubusercontent.com/Zoziologie/ebird2latex/master/e2L.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31677 (31K) [text/plain]
Failed to rename e2L.py to e2L.py.1: (2) No such file or directory
Saving to: ‘e2L.py’

e2L.py              100%[===================>]  30.93K  --.-KB/s    in 0.01s   

2023-11-14 22:53:56 (2.62 MB/s) - ‘e2L.py’ saved [31677/31677]

--2023-11-14 22:54:04--  https://raw.githubusercontent.com/Zoziologie/ebird2latex/master/assets/templateLaTeX.tex
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting

Define your project name


In [2]:
projet_name = "Florida"
filename = projet_name.replace(" ", "_").replace("'", "").replace(".", "")

## 2. Authentification

To be able to download barchat data, you will need to use your eBird login. Mine are stored in a private file on google drive as json file:

```{json}
{
    "username": "your_ebird_username",
    "password": "your_ebird_password"
}
```


In [3]:
from google.colab import drive

drive.mount("/content/drive")
with open("/content/drive/MyDrive/auth.json") as f:
    auth = json.load(f)

Mounted at /content/drive


You can also just type them manually for each session


In [ ]:
%%script false
auth = {
    'username':'your_ebird_username',
    'password':'your_ebird_password'
    }

This allows to load the filw and run [`e2L.auth`](https://github.com/Zoziologie/ebird2latex/blob/caabd2c2a06c9f882d80f637011f26835b792099/e2L.py#L63) to starts a [`requests`](https://docs.python-requests.org/en/latest/) session with your login and password and returns it to you.


In [4]:
session = e2L.auth(auth["username"], auth["password"])

request sessionID from login...
...login succesfull


## 2. Create the bird list

The checklist can be generated for any location with a valid locID code. This can be country, subregion1, subregion2 or specific hotspot. Check out [the old API for a complete list of regions and hotspot](https://confluence.cornell.edu/display/CLOISAPI/eBird-1.1-HotSpotsByRegion) or [the more brief but new API](https://documenter.getpostman.com/view/664302/S1ENwy59#f4f59f90-854e-4ba6-8207-323a8cf0bfe0).

[`e2L.bird_creator`](https://github.com/Zoziologie/ebird2latex/blob/caabd2c2a06c9f882d80f637011f26835b792099/e2L.py#L85) do the following two steps:

- First load the barchart data ([`load_barchart`](https://github.com/Zoziologie/ebird2latex/blob/caabd2c2a06c9f882d80f637011f26835b792099/e2L.py#L130)) from eBird using: `http://ebird.org/barchartData?r={code_loc}&bmo={bmonth}&emo={emonth}&byr={byear}&eyr={eyear}`. This includes the list of bird recorded at this location with its occurance per week.
- Then load the taxonomy information ([`load_taxa`](https://github.com/Zoziologie/ebird2latex/blob/caabd2c2a06c9f882d80f637011f26835b792099/e2L.py#L173)) from the [eBird API](https://documenter.getpostman.com/view/664302/S1ENwy59#952a4310-536d-4ad1-8f3e-77cfb624d1bc). You can choose [any available languages](https://support.ebird.org/en/support/solutions/articles/48000804865-bird-names-in-ebird) and [species category ](https://ebird.org/science/use-ebird-data/the-ebird-taxonomy).


In [5]:
code_loc = "US-FL"
lang = ["EN"]
cat = ["species"]  # domestic,form,hybrid,intergrade,issf,slash,species,spuh
byear = 1900
eyear = 2050
bmonth = 1
emonth = 12
info, bird_list = e2L.bird_creator(
    session, code_loc, lang, cat, byear, eyear, bmonth, emonth
)

Load barchart data from http://ebird.org/barchartData?r=US-FL&bmo=1&emo=12&byr=1900&eyr=2050&fmt=tsv
Load taxonomy for EN


## 3. Add Status

**New.** You can now add a `status` to each species. For the moment, `status` can

- an eBird target for a particular time (`year` or `life`) and place(e.g. `world`, `code_loc` or any other region). [To compute this](https://github.com/Zoziologie/ebird2latex/blob/7052c2db9781dbcfb3ef83881659d539495bf2d1/e2L.py#L210), we check that the species is absent from your list for that region/time (`https://ebird.org/lifelist/{target_loc}/?time={target_time}`).
- `Endemic` or `introduced` according to [Avibase](https://avibase.bsc-eoc.org/). [To compute this](https://github.com/Zoziologie/ebird2latex/blob/7052c2db9781dbcfb3ef83881659d539495bf2d1/e2L.py#L240), we check the value on the avibase checklist for a particular region (`https://avibase.bsc-eoc.org/checklist.jsp?region={code_loc}&list=clements`). Note that this `code_loc` might be difference than eBird.

The first input of the status functions is the label attributed to the species if the condition is true. This label will be needed later when displaying it. Thus, they need to be unique.


In [6]:
target_time = "life"
bird_list = e2L.statusInList("target-world", bird_list, session, "world", target_time)
bird_list = e2L.statusInList("target-US", bird_list, session, "US", target_time)
bird_list = e2L.statusInList(
    "target-code_loc", bird_list, session, code_loc, target_time
)
bird_list = e2L.statusAvibase(["endemic-US", "introduce-US"], bird_list, "USfl")

Load target data from https://ebird.org/lifelist/world/?time=life
Load target data from https://ebird.org/lifelist/US/?time=life
Load target data from https://ebird.org/lifelist/US-FL/?time=life
Load endemic data from https://avibase.bsc-eoc.org/checklist.jsp?region=USfl&list=clements


## 4. Define column

In this next step, we can define what to display in the checklist. This is performed by creating a list of `TableInput(type,option1,option1,option2,option3)` of the following possible `type`:

- `lang`: display the name of the species in a particular language. `option1` can only be one of the language code requested when building the bird list. (latin is always possible (`LA`)). See below for `option2`.
- `freq`: disply occurence frequence. `option1` can be 'year', 'season', 'month' or 'week'. `option2` is the index of `option1` to display (e.g, `month,0` is january, `season,1` is summer). `year` does not have an index.
- `checkbox` display a checkbox. `option1` is the number of checkbox to display (default: 3).
- `line` display empty lines ("\_\_\_") . `option1` is the number of lines (default: 3) and `option2` their length with unit (default: '3ex').
- `note` display note line ("...")length (eg. : 4cm).

**New**. Using `status`, we can now highlight specific species in the checklist. This is done by addinng a `statusTable` to `option2` to the type `lang`. `statusTable` is a list of lists of status criteria and style corresponding. The status criteria to apply should be the same value given when creating the status (first input of the status function). The style are currently:

- `bold`, `italic`, `underline`
- coloring the font with `color-{color_value}`, see [`xcolor`](https://steeven9.github.io/USI-LaTeX/html/packages_hyperref_babel_xcolor3.html) for the list of possible color
- adding a symbol after the name. with `sym-{symbole_value}`. Any single character symbol should work.

Note that the order of the column in `col`define the order in the checklist


In [19]:
col = []
col.append(e2L.TableInput(info, "checkbox", 1))
# col.append(e2L.TableInput(info,'line',1,'8ex'))
# col.append(e2L.TableInput(info,'note','3cm'))
statusTable = [
    ["endemic-US", "bold"],
    ["introduce-US", "color-violet"],
    ["target-world", "sym-*"],
    ["target-US", "sym-*"],
]
col.append(e2L.TableInput(info, "lang", "EN", statusTable))
# col.append(e2L.TableInput(info,'lang','LA'))
# col.append(e2L.TableInput(info,'freq','year'))
# col.append(e2L.TableInput(info,'freq','season',0))
col.append(e2L.TableInput(info, "freq", "month", 4))

## 5. Create the tex file

When creating the $\LaTeX$ document, you can add more formating option to the document (`format`) and change the spacing between row (`spacing`).

You can also choose to display the family name or not `family`.

You can filter the specie to show in the table as you want in `condition_tableau`. The first value is the text displayed and the second is the actual code evaluated for each `bird` from the bird list.
Similarly, you can also display a short rare species list at the end of your checklist with `condition_rare`. To not display this rare table, simply set an impossible condition


In [20]:
format = "letterpaper,margin=.3in,twocolumn"  #'a4paper,margin=15mm,twocolumn'
spacing = ".95"
family = True
condition_tableau = [
    "Main table display only non-hybrid birds with occurence >0.1\\%.",
    " ( bird['freq']['year'] >= 0.001)",
]
# condition_rare = ['\\footnotesize{>.1\\%}'," (bird['freq']['year'] < .01) and (bird['freq']['year'] > 0.001)"]
condition_rare = [
    "\\footnotesize{>.1\\%}",
    " (bird['freq']['year'] < .00) and (bird['freq']['year'] > 0.001)",
]

e2L.write_to_latex(
    projet_name,
    filename,
    bird_list,
    col,
    condition_tableau,
    condition_rare,
    family,
    format,
    spacing,
    info,
)

## 7. Running $\LaTeX$

Install basic tex.


In [ ]:
!sudo apt-get install texlive-latex-recommended texlive-latex-extra

Doing the magic 🤩


In [ ]:
!pdflatex -output-directory=./latex ./latex/Florida.tex